<a href="https://colab.research.google.com/github/GunroarCannon/E-Commerce-sql-streamlit-data-analysis/blob/main/E-Commerce-filesystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Install required packages in Google Colab
!pip install pandas numpy sqlalchemy matplotlib dill pyngrok seaborn plotly
!pip install streamlit streamlit-authenticator
!pip install scikit-learn xgboost
!pip install requests beautifulsoup4

# Import essential libraries
import pandas as pd
import numpy as np
import sqlite3
from sqlalchemy import create_engine, inspect, MetaData, Table, Column, Integer, String, Float, DateTime, ForeignKey, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import json
import requests
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All packages installed successfully!")
print("📊 Ready to build your e-commerce data warehouse!")

✅ All packages installed successfully!
📊 Ready to build your e-commerce data warehouse!


In [16]:
%%writefile dataloader.py
# Data Acquisition - Multiple E-commerce Dataset Options
import requests
import pandas as pd
import zipfile
import os
from io import StringIO

class EcommerceDataAcquisition:
    def __init__(self):
        self.datasets = {}

    def download_sample_ecommerce_data(self):
        """
        Download sample e-commerce datasets from various sources
        """
        print("📥 Downloading E-commerce datasets...")

        # Option 1: UK E-commerce data (Online Retail dataset)
        # Yes, it exists. Should not fail
        try:
            url1 = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"

            #too fictional and short
            #    "https://raw.githubusercontent.com/paulsamuel-w-e/E-commerce-Customer-Behaviour-Dataset/refs/heads/main/E-commerce.csv"
            #no price
            #    https://raw.githubusercontent.com/gilangnr/e-commerce_public_dataset/refs/heads/main/all_data.csv"
            #too limited
            #    "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"
            print("Downloading Online Retail dataset...")
            uk_data = pd.read_excel(url1, engine='openpyxl')
            self.datasets['retail'] = uk_data
            print(f"✅ Retail data: {uk_data.shape}")
        except Exception as e:
            print(f"❌ Retail download failed: {e}")

        # Option 2: Create synthetic comprehensive e-commerce data
        # not needed
        # self.create_synthetic_ecommerce_data()

        return self.datasets

    def create_synthetic_ecommerce_data(self):
        """
        Create realistic synthetic e-commerce data for the warehouse


        Just incase. It's a cool function.
        """
        print("🏭 Generating synthetic e-commerce data...")

        np.random.seed(42)

        # Generate customers data
        num_customers = 5000
        customers = pd.DataFrame({
            'customer_id': range(1, num_customers + 1),
            'first_name': np.random.choice(['John', 'Jane', 'Mike', 'Sarah', 'David', 'Lisa', 'Chris', 'Emma'], num_customers),
            'last_name': np.random.choice(['Smith', 'Johnson', 'Brown', 'Davis', 'Wilson', 'Moore', 'Taylor', 'Anderson'], num_customers),
            'email': [f'user{i}@email.com' for i in range(1, num_customers + 1)],
            'registration_date': pd.date_range('2020-01-01', '2024-01-01', periods=num_customers),
            'city': np.random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia'], num_customers),
            'state': np.random.choice(['NY', 'CA', 'IL', 'TX', 'AZ', 'PA'], num_customers),
            'country': 'USA',
            'customer_segment': np.random.choice(['Premium', 'Standard', 'Basic'], num_customers, p=[0.2, 0.5, 0.3])
        })

        # Generate products data
        num_products = 1000
        categories = ['Electronics', 'Clothing', 'Home & Garden', 'Books', 'Sports', 'Beauty', 'Toys']
        products = pd.DataFrame({
            'product_id': range(1, num_products + 1),
            'product_name': [f'Product {i}' for i in range(1, num_products + 1)],
            'category': np.random.choice(categories, num_products),
            'supplier_id': np.random.randint(1, 101, num_products),
            'unit_price': np.round(np.random.exponential(50) + 10, 2),
            'cost_price': lambda x: x * 0.7,  # Will be calculated
            'launch_date': pd.date_range('2020-01-01', '2023-12-31', periods=num_products),
            'brand': np.random.choice(['Brand A', 'Brand B', 'Brand C', 'Brand D', 'Brand E'], num_products)
        })
        products['cost_price'] = np.round(products['unit_price'] * 0.7, 2)

        # Generate suppliers data
        num_suppliers = 100
        suppliers = pd.DataFrame({
            'supplier_id': range(1, num_suppliers + 1),
            'supplier_name': [f'Supplier {i}' for i in range(1, num_suppliers + 1)],
            'contact_email': [f'supplier{i}@company.com' for i in range(1, num_suppliers + 1)],
            'country': np.random.choice(['USA', 'China', 'Germany', 'Japan', 'India'], num_suppliers),
            'rating': np.round(np.random.uniform(3.0, 5.0, num_suppliers), 1)
        })

        # Generate sales transactions
        num_transactions = 50000
        transactions = pd.DataFrame({
            'transaction_id': range(1, num_transactions + 1),
            'customer_id': np.random.randint(1, num_customers + 1, num_transactions),
            'product_id': np.random.randint(1, num_products + 1, num_transactions),
            'order_date': pd.date_range('2022-01-01', '2024-09-01', periods=num_transactions),
            'quantity': np.random.randint(1, 6, num_transactions),
            'channel': np.random.choice(['Website', 'Mobile App', 'Store', 'Social Media'], num_transactions, p=[0.4, 0.3, 0.2, 0.1]),
            'payment_method': np.random.choice(['Credit Card', 'PayPal', 'Debit Card', 'Bank Transfer'], num_transactions),
            'shipping_cost': np.round(np.random.uniform(0, 15, num_transactions), 2),
            'discount_applied': np.random.choice([0, 5, 10, 15, 20], num_transactions, p=[0.5, 0.2, 0.15, 0.1, 0.05])
        })

        # Calculate revenue
        product_prices = dict(zip(products['product_id'], products['unit_price']))
        transactions['unit_price'] = transactions['product_id'].map(product_prices)
        transactions['gross_revenue'] = transactions['quantity'] * transactions['unit_price']
        transactions['discount_amount'] = transactions['gross_revenue'] * transactions['discount_applied'] / 100
        transactions['net_revenue'] = transactions['gross_revenue'] - transactions['discount_amount']

        # Generate marketing campaigns
        campaigns = pd.DataFrame({
            'campaign_id': range(1, 51),
            'campaign_name': [f'Campaign {i}' for i in range(1, 51)],
            'start_date': pd.date_range('2022-01-01', '2024-06-01', periods=50),
            'end_date': pd.date_range('2022-02-01', '2024-07-01', periods=50),
            'budget': np.round(np.random.uniform(1000, 50000, 50), 2),
            'channel': np.random.choice(['Google Ads', 'Facebook', 'Email', 'Instagram', 'YouTube'], 50),
            'target_segment': np.random.choice(['Premium', 'Standard', 'Basic', 'All'], 50)
        })

        # Store all datasets
        self.datasets.update({
            'customers': customers,
            'products': products,
            'suppliers': suppliers,
            'transactions': transactions,
            'campaigns': campaigns
        })

        print("✅ Synthetic data generated successfully!")
        for name, df in self.datasets.items():
            print(f"   📊 {name}: {df.shape}")

        return self.datasets

    def save_datasets_to_csv(self, folder_path="data"):
        """Save all datasets to CSV files"""
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        for name, df in self.datasets.items():
            file_path = f"{folder_path}/{name}.csv"
            df.to_csv(file_path, index=False)
            print(f"💾 Saved {name} to {file_path}")

# Initialize and run data acquisition
data_acquisition = EcommerceDataAcquisition()
datasets = data_acquisition.download_sample_ecommerce_data()

# Display sample data
print("\n📋 Sample data preview:")
for name, df in datasets.items():
    print(f"\n--- {name.upper()} ---")
    print(df.head(3))
    print(f"Shape: {df.shape}")

main_dataset = datasets["retail"]


Writing dataloader.py


In [27]:
%%writefile schema.py

from dataloader import *

# Database Schema Design - Dimensional Modeling for E-commerce Data Warehouse
'''
Uses sqlalchemy to create_engine for dimensional models.
Base class gotten from ext.declarative is the base for model definitions.
'''
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime, Date, Boolean, ForeignKey, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import sqlite3

Base = declarative_base()
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, ForeignKey, Boolean
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

class EcommerceDimensionalModel:
    """
    Comprehensive dimensional model for e-commerce data warehouse
    Following Kimball methodology with fact and dimension tables.

    Where dimensions are descriptive and facts are statistics/numeric verbs.
    """
    def __init__(self, db_path="ecommerce_warehouse.db"):
        self.db_path = db_path
        self.engine = create_engine(f"sqlite:///{db_path}")

        Base.metadata.drop_all(self.engine)
        Base.metadata.create_all(self.engine)
    # DIMENSION TABLES
    class DimCustomer(Base):
        __tablename__ = 'dim_customer'
        customer_key = Column(Integer, primary_key=True, autoincrement=True)
        customer_id = Column(Integer, unique=True, nullable=False)
        country = Column(String(100))

    class DimProduct(Base):
        __tablename__ = 'dim_product'
        product_key = Column(Integer, primary_key=True, autoincrement=True)
        stock_code = Column(String(50), unique=True, nullable=False)
        description = Column(String(255))

    class DimDate(Base):
        __tablename__ = 'dim_date'
        date_key = Column(Integer, primary_key=True)  # YYYYMMDD format
        full_date = Column(Date, unique=True, nullable=False)
        year = Column(Integer)
        month = Column(Integer)
        day = Column(Integer)
        week = Column(Integer)
        weekday = Column(String(20))
        is_weekend = Column(Boolean)

    # FACT TABLE
    class FactSales(Base):
        __tablename__ = 'fact_sales'
        sales_key = Column(Integer, primary_key=True, autoincrement=True)
        invoice_no = Column(String(50))
        date_key = Column(Integer, ForeignKey('dim_date.date_key'), nullable=False)
        customer_key = Column(Integer, ForeignKey('dim_customer.customer_key'))
        product_key = Column(Integer, ForeignKey('dim_product.product_key'))
        quantity = Column(Integer)
        unit_price = Column(Float)
        revenue = Column(Float)  # Quantity * UnitPrice

    def create_all_tables(self):
        Base.metadata.create_all(self.engine)
        print("✅ Tables created successfully!")


    def get_table_info(self):
        """Get information about all tables"""
        inspector = inspect(create_engine(f'sqlite:///{self.db_path}'))

        print("📋 Database Tables:")
        for table in inspector.get_table_names():
            print(f"   📊 {table}")

        return inspector

# Create the dimensional model
print("🏭 Initializing E-commerce Dimensional Model...")
warehouse = EcommerceDimensionalModel()
warehouse.create_all_tables()
warehouse.get_table_info()

print("""
🎯 Dimensional Model Summary:

DIMENSION TABLES:
• dim_customer - Customer information (CustomerID, Country)
• dim_product - Product catalog (StockCode, Description)
• dim_date - Date dimension (Year, Month, Day, Week, Weekday)

FACT TABLES:
• fact_sales - Transaction-level sales data (InvoiceNo, Quantity, UnitPrice, Revenue)

🔑 Key Features:
✓ Star schema design with surrogate keys
✓ Date dimension for time-series analysis
✓ Supports sales & revenue analytics
✓ Ready for customer, product & time-based queries
✓ Marketing attribution modeling
""")


Overwriting schema.py


In [18]:
%%writefile model.py

from dataloader import *
from schema import *
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy.orm import sessionmaker
from sqlalchemy import text
import logging
import time

class EcommerceETL:
    def __init__(self, model_class):
        self.model_class = model_class
        self.engine = model_class.engine
        self.Session = sessionmaker(bind=self.engine)

        logging.basicConfig(level=logging.INFO, force=True)
        self.logger = logging.getLogger(__name__)

    def extract_raw_data(self, datasets):
        """Extract raw data from given sources"""
        self.raw_data = datasets
        print("yes")
        self.logger.info(f"✅ Extracted datasets: {list(datasets.keys())}")
        return datasets

    def transform_and_load_dimensions(self):
        """Transform and load dimensions into DB"""
        session = self.Session()
        try:
            self._load_dim_date(session)
            self._load_dim_customer(session)
            self._load_dim_product(session)
            session.commit()

            self._load_fact_sales(session)


            session.commit()
            self.logger.info("✅ Dimensions loaded successfully!")
        except Exception as e:
            session.rollback()
            self.logger.error(f"❌ ETL Error: {e}")
        finally:
            session.close()


    def _load_fact_sales_old(self, session):
        self.logger.info("💰 Loading fact_sales...")
        fact_sales = []
         # Prepare mappings from raw IDs to surrogate keys
        customer_map = pd.read_sql("SELECT customer_key, customer_id FROM dim_customer", self.model_class.engine)
        product_map = pd.read_sql("SELECT product_key, stock_code FROM dim_product", self.model_class.engine)

        #dicts for faster look up

        customer_dict = dict(zip(customer_map["customer_id"], customer_map["customer_key"]))
        product_dict = dict(zip(product_map["stock_code"], product_map["product_key"]))


        '''
        TODO: Makes it faster
transactions_df['InvoiceDate'] = pd.to_datetime(transactions_df['InvoiceDate'])
transactions_df['date_key'] = transactions_df['InvoiceDate'].dt.strftime('%Y%m%d').astype(int)

# Map surrogate keys using vectorized pandas map
transactions_df['customer_key'] = transactions_df['CustomerID'].map(customer_dict)
transactions_df['product_key'] = transactions_df['StockCode'].map(product_dict)

# Calculate revenue
transactions_df['revenue'] = transactions_df['Quantity'] * transactions_df['UnitPrice']

        '''
        for i, row in self.raw_data["transactions"].iterrows():
          fact_sales.append({
              "sales_key": i,
              "invoice_no": row["InvoiceNo"],
              "date_key": int(pd.to_datetime(row["InvoiceDate"]).strftime('%Y%m%d')),
              "customer_key": customer_dict.get(row["CustomerID"]),
              "product_key": product_dict.get(row["StockCode"]),
              "quantity": row["Quantity"],
              "unit_price": row["UnitPrice"],
              "revenue": row["Quantity"]*row["UnitPrice"]
          })

        session.bulk_insert_mappings(self.model_class.FactSales, fact_sales)
        self.logger.info(f"✅ {len(fact_sales)} fact records loaded")
    def _load_fact_sales(self, session): #   def _load_fact_sales(self, session):
        self.logger.info("💰 Loading fact_sales (vectorized)...")

        # Prepare mappings from raw IDs to surrogate keys
        customer_map = pd.read_sql(
            "SELECT customer_key, customer_id FROM dim_customer",
            self.model_class.engine
        )
        product_map = pd.read_sql(
            "SELECT product_key, stock_code FROM dim_product",
            self.model_class.engine
        )
        self.logger.info(product_map.head(3))

        transactions = self.raw_data["transactions"].copy()
        # 1. Normalize transactions columns
        transactions['StockCode'] = transactions['StockCode'].astype(str).str.strip().str.upper()
        transactions['CustomerID'] = transactions['CustomerID'].astype(pd.Int64Dtype())  # nullable integer

        # 2. Normalize dimension maps
        product_map['stock_code'] = product_map['stock_code'].astype(str).str.strip().str.upper()
        customer_map['customer_id'] = customer_map['customer_id'].astype(int)

        # 3. Build mapping dicts
        product_dict = dict(zip(product_map['stock_code'], product_map['product_key']))
        customer_dict = dict(zip(customer_map['customer_id'], customer_map['customer_key']))

        # 4. Map keys
        transactions['product_key'] = transactions['StockCode'].map(product_dict)
        transactions['customer_key'] = transactions['CustomerID'].map(customer_dict)

        count = 0
        print(transactions['StockCode'].head(3))
        print(product_map.head(3))
        for i in transactions['StockCode']:
            if i not in product_dict.keys():
                print("Missing STOCK CODE",i)
                count += 1
                if count > 10:
                  break

        # 5. Check missing keys
        missing_products = transactions[transactions['product_key'].isna()]['StockCode'].unique()
        missing_customers = transactions[transactions['customer_key'].isna()]['CustomerID'].unique()

        print("Missing product keys:", missing_products)
        print("Missing customer keys:", missing_customers)

        #transactions['product_key'] = transactions['StockCode'].str.strip().str.upper().map(product_dict)
        transactions['date_key'] = pd.to_datetime(transactions['InvoiceDate']).dt.strftime('%Y%m%d').astype(int)
        transactions['revenue'] = transactions['Quantity'] * transactions['UnitPrice']

        # Add a sales_key column (optional, can just use index)
        transactions['sales_key'] = transactions.index

        # Assign dummy keys for missing
        dummy_customer_key = 0
        dummy_product_key = 0

        transactions['customer_key'] = transactions['customer_key'].fillna(dummy_customer_key)
        transactions['product_key'] = transactions['product_key'].fillna(dummy_product_key)

        transactions = transactions.dropna(subset=['customer_key', 'product_key'])
        invalid_stockcodes = set(transactions['StockCode']) - set(product_dict.keys())
        print("StockCodes not in dim_product:", list(invalid_stockcodes)[:20])


        # Keep only the columns needed for the fact table
        fact_sales = transactions[['sales_key', 'InvoiceNo', 'date_key', 'customer_key',
                                   'product_key', 'Quantity', 'UnitPrice', 'revenue']].rename(
            columns={'Quantity': 'quantity', 'UnitPrice': 'unit_price'}
        ).to_dict(orient='records')

        # Bulk insert
        session.bulk_insert_mappings(self.model_class.FactSales, fact_sales)
        self.logger.info(f"✅ {len(fact_sales)} fact records loaded (vectorized)")

    def _load_dim_date(self, session):
        self.logger.info("📅 Loading date dimension...")
        start_date = datetime(2010, 1, 1)
        end_date = datetime(2012, 12, 31)

        records = []
        current_date = start_date
        while current_date <= end_date:
            records.append({
                "date_key": int(current_date.strftime('%Y%m%d')),
                "full_date": current_date.date(),
                "year": current_date.year,
                "month": current_date.month,
                "day": current_date.day,
                "week": current_date.isocalendar()[1],
                "weekday": current_date.strftime('%A'),
                "is_weekend": current_date.weekday() >= 5
            })
            current_date += timedelta(days=1)

        session.bulk_insert_mappings(self.model_class.DimDate, records)
        self.logger.info(f"✅ {len(records)} date records loaded")

    def _load_dim_customer(self, session):
        self.logger.info("👥 Loading customer dimension...")
        customers = self.raw_data["customers"]
        records = []
        for i, row in customers.iterrows():
            records.append({
                "customer_id": row["CustomerID"],
                "country": row.get("Country", "Unknown")
            })
        session.bulk_insert_mappings(self.model_class.DimCustomer, records)
        self.logger.info(f"✅ {len(records)} customer records loaded")
    def _load_dim_product(self, session):
        self.logger.info("📦 Loading product dimension...")

        # Original products table
        products = self.raw_data["products"].copy()
        products['StockCode'] = products['StockCode'].astype(str).str.strip().str.upper()
        products['Description'] = products['Description'].fillna("No description")

        # Drop exact duplicates (StockCode + Description)
        products = products.drop_duplicates(subset=['StockCode'])

        # Collect all StockCodes from transactions
        transactions = self.raw_data.get("transactions")
        if transactions is not None:
            transactions_codes = transactions['StockCode'].astype(str).str.strip().str.upper().unique()
            missing_codes = set(transactions_codes) - set(products['StockCode'])
            self.logger.info("Checking missing cause transaction is not null")
            self.logger.info(f"Total transaction StockCodes: {len(transactions_codes)}")
            self.logger.info(f"Total product StockCodes: {len(products['StockCode'])}")
            self.logger.info(f"Missing codes count: {len(missing_codes)}")

            if missing_codes:
                self.logger.info(f"Adding {len(missing_codes)} missing StockCodes to dim_product...")
                missing_df = pd.DataFrame({
                    'StockCode': list(missing_codes),
                    'Description': 'Unknown Product'
                })
                products = pd.concat([products, missing_df], ignore_index=True)

        # Prepare records for bulk insert
        records = []
        for i, row in products.iterrows():
            records.append({
                "stock_code": row["StockCode"],
                "description": row.get("Description", "No description")
            })

        # Bulk insert into dim_product
        session.bulk_insert_mappings(self.model_class.DimProduct, records)
        self.logger.info(f"✅ {len(records)} product records loaded")
        time.sleep(1)

    def run_query(self, sql):
        """
        Run a raw SQL query on the warehouse and return a Pandas DataFrame.
        """
        with self.engine.connect() as conn:
            result = conn.execute(text(sql))
            # Convert results to DataFrame
            df = pd.DataFrame(result.fetchall(), columns=result.keys())
        return df

Base.metadata.drop_all(warehouse.engine)  # Drops old tables
Base.metadata.create_all(warehouse.engine)

# 🔹 Create dimensions from raw data (cleaned) 🔹

# Customers: drop duplicates and missing IDs
customers_df = main_dataset[['CustomerID', 'Country']].dropna(subset=['CustomerID']).drop_duplicates(subset=['CustomerID'])
customers_df['CustomerID'] = customers_df['CustomerID'].astype(int)

# Products: drop duplicates, drop missing StockCode, normalize StockCode, fill missing description
products_df = main_dataset[['StockCode', 'Description']].copy()

# Drop rows where StockCode is missing or empty after stripping
products_df['StockCode'] = products_df['StockCode'].astype(str).str.strip().str.upper()
products_df = products_df[products_df['StockCode'] != '']
products_df = products_df.drop_duplicates(subset=['StockCode'])

# Fill missing descriptions
products_df['Description'] = products_df['Description'].fillna("No description")

# Suppliers: dummy table if none exists
suppliers_df = pd.DataFrame({"SupplierID": [1], "Name": ["Default Supplier"]})

# Transactions: drop duplicates, drop rows with missing critical info, normalize StockCode
transactions_df = main_dataset[['InvoiceNo', 'CustomerID', 'UnitPrice', 'Quantity', 'StockCode', 'InvoiceDate']].dropna()
transactions_df = transactions_df.drop_duplicates()

transactions_df['StockCode'] = transactions_df['StockCode'].astype(str).str.strip().str.upper()

transactions_df['customer_key'] = transactions_df['CustomerID'].map(customer_dict)
transactions_df['product_key'] = transactions_df['StockCode'].map(product_dict)

# Keep only rows with valid keys
transactions_df = transactions_df.dropna(subset=['customer_key', 'product_key'])


# Pass to ETL
datasets = {
    "customers": customers_df,
    "products": products_df,
    "suppliers": suppliers_df,
    "transactions": transactions_df
}

etl = EcommerceETL(warehouse)
etl.extract_raw_data(datasets)
etl.transform_and_load_dimensions()

class SQLQueries:
    def __init__(self, etl):
        self.etl = etl

    # Top N customers by revenue
    def top_n_customers(self, n=10):
        padf = self.etl.run_query(f"""
            SELECT c.customer_id, SUM(f.revenue) AS revenue
            FROM fact_sales f
            JOIN dim_customer c ON f.customer_key = c.customer_key
            GROUP BY c.customer_id
            ORDER BY revenue DESC
            LIMIT {n}
        """)
        return padf

    # Top sales by country
    def top_sales_by_country(self):
        padf = self.etl.run_query(f"""
            SELECT c.country, SUM(f.revenue) AS revenue
            FROM fact_sales f
            JOIN dim_customer c ON f.customer_key = c.customer_key
            GROUP BY c.country
            ORDER BY revenue DESC
        """)
        return padf

    # Top N products by revenue
    def top_n_products(self, n=10):
        padf =  self.etl.run_query(f"""
            SELECT p.description, SUM(f.revenue) AS revenue
            FROM fact_sales f
            JOIN dim_product p ON f.product_key = p.product_key
            GROUP BY p.description
            ORDER BY revenue DESC
            LIMIT {n}
        """)
        return padf

    # Monthly sales trend
    def monthly_sales_trend(self):
        padf = self.etl.run_query(f"""
            SELECT substr(f.date_key,1,6) AS year_month, SUM(f.revenue) AS revenue
            FROM fact_sales f
            GROUP BY year_month
            ORDER BY year_month
        """)
        return padf

    # RFM-style customer summary
    def customer_rfm_summary(self):
        padf = self.etl.run_query(f"""
            SELECT
                f.customer_key,
                MAX(f.date_key) AS last_purchase,
                COUNT(DISTINCT f.invoice_no) AS frequency,
                SUM(f.revenue) AS monetary
            FROM fact_sales f
            GROUP BY f.customer_key
        """)
        return padf


queries = SQLQueries(etl)
customer_map = pd.read_sql(
    "SELECT customer_key, customer_id FROM dim_customer",
    etl.model_class.engine
)
product_map = pd.read_sql(
    "SELECT product_key, stock_code FROM dim_product",
    etl.model_class.engine
)

# dicts for faster lookup
customer_dict = dict(zip(customer_map["customer_id"], customer_map["customer_key"]))
product_dict = dict(zip(product_map["stock_code"], product_map["product_key"]))

print(customer_dict)
print(product_dict)



# Top 10 customers
top_customers = queries.top_n_customers(10)
print(top_customers)

# Top sales by country
country_sales = queries.top_sales_by_country()
print(country_sales)

# Top 10 products
top_products = queries.top_n_products(10)
print(top_products)

# Monthly sales trend
monthly_trend = queries.monthly_sales_trend()
print(monthly_trend)

# Customer RFM summary
rfm_summary = queries.customer_rfm_summary()
print(rfm_summary.head())



Writing model.py


In [25]:

%%writefile app.py

from dataloader import *
from schema import *
from model import *


import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sqlite3
from datetime import datetime, timedelta
import hashlib
import hmac
from sqlalchemy import create_engine
import numpy as np
import dill

# Page Configuration
st.set_page_config(
    page_title="🛍️ E-commerce Analytics Dashboard",
    page_icon="📊",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS for advanced styling
def load_css():
    st.markdown("""
    <style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;500;600;700&display=swap');

    /* Root variables for theming */
    :root {
        --primary-gradient: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        --secondary-gradient: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
        --success-gradient: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%);
        --warning-gradient: linear-gradient(135deg, #fa709a 0%, #fee140 100%);
        --dark-bg: #0e1117;
        --card-bg: rgba(255, 255, 255, 0.05);
        --glass-bg: rgba(255, 255, 255, 0.1);
        --text-primary: #ffffff;
        --text-secondary: rgba(255, 255, 255, 0.7);
        --border-color: rgba(255, 255, 255, 0.1);
    }

    /* Global Styles */
    .main .block-container {
        padding-top: 2rem;
        max-width: 100%;
    }

    body {
        font-family: 'Inter', sans-serif;
        background: var(--dark-bg);
        color: var(--text-primary);
    }

    /* Grid background animation */
    .stApp::before {
        content: '';
        position: fixed;
        top: 0;
        left: 0;
        width: 100%;
        height: 100%;
        background:
            linear-gradient(90deg, rgba(255,255,255,0.02) 1px, transparent 1px),
            linear-gradient(rgba(255,255,255,0.02) 1px, transparent 1px);
        background-size: 50px 50px;
        animation: gridMove 20s linear infinite;
        pointer-events: none;
        z-index: -1;
    }

    @keyframes gridMove {
        0% { transform: translate(0, 0); }
        100% { transform: translate(50px, 50px); }
    }

    /* Header styling */
    .main-header {
        background: var(--primary-gradient);
        padding: 2rem 3rem;
        border-radius: 20px;
        margin-bottom: 2rem;
        box-shadow: 0 20px 40px rgba(102, 126, 234, 0.3);
        backdrop-filter: blur(20px);
        border: 1px solid rgba(255, 255, 255, 0.1);
    }

    .main-header h1 {
        color: white;
        font-size: 2.5rem;
        font-weight: 700;
        margin-bottom: 0.5rem;
        text-shadow: 0 2px 10px rgba(0,0,0,0.3);
    }

    .main-header p {
        color: rgba(255, 255, 255, 0.9);
        font-size: 1.1rem;
        font-weight: 400;
        margin: 0;
    }

    /* Metric cards */
    .metric-card {
        background: var(--glass-bg);
        backdrop-filter: blur(20px);
        border: 1px solid var(--border-color);
        border-radius: 16px;
        padding: 1.5rem;
        margin: 0.5rem 0;
        box-shadow: 0 8px 32px rgba(0, 0, 0, 0.3);
        transition: all 0.3s ease;
        position: relative;
        overflow: hidden;
    }

    .metric-card::before {
        content: '';
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        height: 3px;
        background: var(--primary-gradient);
    }

    .metric-card:hover {
        transform: translateY(-5px);
        box-shadow: 0 15px 45px rgba(0, 0, 0, 0.4);
        border-color: rgba(255, 255, 255, 0.2);
    }

    .metric-value {
        font-size: 2.2rem;
        font-weight: 700;
        color: var(--text-primary);
        margin-bottom: 0.5rem;
    }

    .metric-label {
        font-size: 0.9rem;
        color: var(--text-secondary);
        font-weight: 500;
        text-transform: uppercase;
        letter-spacing: 0.5px;
    }

    .metric-change {
        font-size: 0.8rem;
        font-weight: 600;
        padding: 0.2rem 0.5rem;
        border-radius: 12px;
        margin-top: 0.5rem;
        display: inline-block;
    }

    .metric-change.positive {
        background: rgba(76, 175, 80, 0.2);
        color: #4CAF50;
        border: 1px solid rgba(76, 175, 80, 0.3);
    }

    .metric-change.negative {
        background: rgba(244, 67, 54, 0.2);
        color: #f44336;
        border: 1px solid rgba(244, 67, 54, 0.3);
    }

    /* Sidebar styling */
    .css-1d391kg {
        background: var(--glass-bg);
        backdrop-filter: blur(20px);
        border-right: 1px solid var(--border-color);
    }

    /* Filter containers */
    .filter-container {
        background: var(--card-bg);
        backdrop-filter: blur(10px);
        border: 1px solid var(--border-color);
        border-radius: 12px;
        padding: 1rem;
        margin: 1rem 0;
    }

    /* Chart containers */
    .chart-container {
        background: var(--card-bg);
        backdrop-filter: blur(10px);
        border: 1px solid var(--border-color);
        border-radius: 16px;
        padding: 1.5rem;
        margin: 1rem 0;
        box-shadow: 0 8px 32px rgba(0, 0, 0, 0.1);
    }

    /* Custom button styling */
    .stButton > button {
        background: var(--primary-gradient);
        color: white;
        border: none;
        border-radius: 12px;
        padding: 0.5rem 1.5rem;
        font-weight: 600;
        transition: all 0.3s ease;
        box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3);
    }

    .stButton > button:hover {
        transform: translateY(-2px);
        box-shadow: 0 8px 25px rgba(102, 126, 234, 0.4);
    }

    /* Login form styling */
    .login-form {
        max-width: 400px;
        margin: 0 auto;
        padding: 2rem;
        background: var(--glass-bg);
        backdrop-filter: blur(20px);
        border: 1px solid var(--border-color);
        border-radius: 20px;
        box-shadow: 0 20px 40px rgba(0, 0, 0, 0.3);
    }

    .login-header {
        text-align: center;
        margin-bottom: 2rem;
    }

    .login-header h2 {
        background: var(--primary-gradient);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
        background-clip: text;
        font-size: 2rem;
        font-weight: 700;
        margin-bottom: 0.5rem;
    }

    /* Success/Error alerts */
    .alert {
        padding: 1rem;
        border-radius: 12px;
        margin: 1rem 0;
        backdrop-filter: blur(10px);
    }

    .alert-success {
        background: rgba(76, 175, 80, 0.1);
        border: 1px solid rgba(76, 175, 80, 0.3);
        color: #4CAF50;
    }

    .alert-error {
        background: rgba(244, 67, 54, 0.1);
        border: 1px solid rgba(244, 67, 54, 0.3);
        color: #f44336;
    }

    /* Responsive design */
    @media (max-width: 768px) {
        .main-header {
            padding: 1.5rem;
        }

        .main-header h1 {
            font-size: 2rem;
        }

        .metric-value {
            font-size: 1.8rem;
        }
    }

    /* Loading animation */
    .loading-spinner {
        width: 40px;
        height: 40px;
        border: 4px solid rgba(102, 126, 234, 0.3);
        border-top: 4px solid #667eea;
        border-radius: 50%;
        animation: spin 1s linear infinite;
        margin: 20px auto;
    }

    @keyframes spin {
        0% { transform: rotate(0deg); }
        100% { transform: rotate(360deg); }
    }

    /* Hide Streamlit branding */
    #MainMenu {visibility: hidden;}
    footer {visibility: hidden;}
    header {visibility: hidden;}

    </style>
    """, unsafe_allow_html=True)

# Authentication functions
def hash_password(password):
    """Hash a password for storing."""
    return hashlib.sha256(str.encode(password)).hexdigest()

def check_password(stored_password, provided_password):
    """Verify a stored password against provided password."""
    return stored_password == hash_password(provided_password)

def init_auth():
    """Initialize authentication state."""
    if 'authenticated' not in st.session_state:
        st.session_state.authenticated = False
    if 'username' not in st.session_state:
        st.session_state.username = ""

# User database (in production, use a proper database)
USERS = {
    "admin": hash_password("admin123"),
    "analyst": hash_password("analyst123"),
    "manager": hash_password("manager123"),
    "demo": hash_password("demo123")
}

def login_page():
    """Display login page."""
    st.markdown('<div class="login-form">', unsafe_allow_html=True)

    st.markdown("""
        <div class="login-header">
            <h2>🛍️ E-commerce Analytics</h2>
            <p style="color: rgba(255,255,255,0.7);">Sign in to access your dashboard</p>
        </div>
    """, unsafe_allow_html=True)

    with st.form("login_form"):
        username = st.text_input("Username", placeholder="Enter your username")
        password = st.text_input("Password", type="password", placeholder="Enter your password")
        submit_button = st.form_submit_button("🔐 Sign In", use_container_width=True)

        if submit_button:
            if username in USERS and check_password(USERS[username], password):
                st.session_state.authenticated = True
                st.session_state.username = username
                st.success("✅ Login successful!")
                st.rerun()
            else:
                st.error("❌ Invalid username or password")

    st.markdown('</div>', unsafe_allow_html=True)

    # Demo credentials
    st.markdown("""
        <div style="margin-top: 2rem; padding: 1rem; background: rgba(255,255,255,0.05); border-radius: 12px;">
            <h4 style="margin-bottom: 1rem;">📝 Demo Credentials:</h4>
            <div style="font-family: monospace; font-size: 0.9rem;">
                <strong>Username:</strong> demo<br>
                <strong>Password:</strong> demo123
            </div>
        </div>
    """, unsafe_allow_html=True)

# Initialize your database connection and classes here
@st.cache_resource
def init_database():
    """Initialize database connection and ETL classes."""
    # Replace with your actual database initialization
    try:
        # warehouse = EcommerceDimensionalModel()
        # etl = EcommerceETL(warehouse)
        queries = SQLQueries(etl)
        # return warehouse, etl, queries

        # For demo purposes, using dummy data
        return warehouse, etl, queriesn
    except Exception as e:
        st.error(f"Database initialization failed: {e}")
        return None, None, None

# Sample data for demonstration (replace with your actual data)
@st.cache_data
def load_sample_data():
    """Load sample data for demonstration."""

    try:
        # Sample sales data
        dates = pd.date_range('2023-01-01', '2024-12-31', freq='D')
        np.random.seed(42)

        sales_data = pd.DataFrame({
            'date': dates,
            'revenue': np.random.normal(10000, 2000, len(dates)).clip(min=0),
            'orders': np.random.poisson(50, len(dates)),
            'customers': np.random.poisson(30, len(dates))
        })

        # Ensure date column is datetime
        sales_data['date'] = pd.to_datetime(sales_data['date'])

        # Sample country data
        countries = ['United Kingdom', 'Germany', 'France', 'Netherlands', 'Belgium', 'Switzerland', 'Austria', 'Italy']
        country_data = pd.DataFrame({
            'country': countries,
            'revenue': np.random.uniform(50000, 500000, len(countries)),
            'customers': np.random.randint(100, 1000, len(countries))
        })

        # Sample product data
        products = [f'Product {i}' for i in range(1, 21)]
        product_data = pd.DataFrame({
            'product': products,
            'revenue': np.random.uniform(10000, 100000, len(products)),
            'quantity': np.random.randint(100, 2000, len(products))
        })

        return sales_data, country_data, product_data

    except Exception as e:
        st.error(f"Error loading sample data: {e}")
        # Return empty dataframes as fallback
        return pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

def create_metric_card(title, value, change=None, format_func=None):
    """Create a styled metric card."""
    if format_func:
        formatted_value = format_func(value)
    else:
        formatted_value = f"{value:,.0f}" if isinstance(value, (int, float)) else str(value)

    change_html = ""
    if change is not None:
        change_class = "positive" if change >= 0 else "negative"
        change_symbol = "↗️" if change >= 0 else "↘️"
        change_html = f'<div class="metric-change {change_class}">{change_symbol} {abs(change):.1f}%</div>'

    st.markdown(f"""
        <div class="metric-card">
            <div class="metric-value">{formatted_value}</div>
            <div class="metric-label">{title}</div>
            {change_html}
        </div>
    """, unsafe_allow_html=True)

def create_charts(sales_data, country_data, product_data, date_range, selected_countries):
    """Create all dashboard charts."""

    try:
        # Convert date_range to datetime for comparison
        if len(date_range) == 2:
            start_date = pd.to_datetime(date_range[0])
            end_date = pd.to_datetime(date_range[1])
            # Filter data based on selections
            filtered_sales = sales_data[
                (sales_data['date'] >= start_date) &
                (sales_data['date'] <= end_date)
            ]
        else:
            filtered_sales = sales_data

        if filtered_sales.empty:
            st.warning("No data available for the selected date range.")
            return None, None, None, None

        filtered_countries = country_data[country_data['country'].isin(selected_countries)]

        # 1. Sales Trend Chart
        fig_trend = go.Figure()
        fig_trend.add_trace(go.Scatter(
            x=filtered_sales['date'],
            y=filtered_sales['revenue'],
            mode='lines',
            name='Daily Revenue',
            line=dict(color='#667eea', width=3),
            fill='tonexty',
            fillcolor='rgba(102, 126, 234, 0.1)'
        ))

        fig_trend.update_layout(
            title="📈 Sales Trend Over Time",
            xaxis_title="Date",
            yaxis_title="Revenue ($)",
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            font=dict(color='white'),
            title_font_size=16,
            showlegend=False,
            xaxis=dict(gridcolor='rgba(255,255,255,0.1)'),
            yaxis=dict(gridcolor='rgba(255,255,255,0.1)')
        )

        # 2. Revenue by Country
        fig_country = px.bar(
            filtered_countries.head(10),
            x='country',
            y='revenue',
            title="🌍 Revenue by Country (Top 10)",
            color='revenue',
            color_continuous_scale='viridis'
        )
        fig_country.update_layout(
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            font=dict(color='white'),
            title_font_size=16,
            xaxis=dict(gridcolor='rgba(255,255,255,0.1)'),
            yaxis=dict(gridcolor='rgba(255,255,255,0.1)')
        )

        # 3. Top Products
        fig_products = px.pie(
            product_data.head(8),
            values='revenue',
            names='product',
            title="🏆 Top Products by Revenue",
            color_discrete_sequence=px.colors.qualitative.Set3
        )
        fig_products.update_layout(
            plot_bgcolor='rgba(0,0,0,0)',
            paper_bgcolor='rgba(0,0,0,0)',
            font=dict(color='white'),
            title_font_size=16,
            showlegend=True,
            legend=dict(
                orientation="v",
                yanchor="middle",
                y=0.5,
                xanchor="left",
                x=1.01
            )
        )

        # 4. Monthly Revenue Heatmap
        monthly_data = filtered_sales.copy()
        monthly_data['month'] = monthly_data['date'].dt.month
        monthly_data['year'] = monthly_data['date'].dt.year
        monthly_pivot = monthly_data.groupby(['year', 'month'])['revenue'].sum().unstack(fill_value=0)

        if not monthly_pivot.empty:
            fig_heatmap = px.imshow(
                monthly_pivot.values,
                x=[f"M{i}" for i in range(1, 13)],
                y=monthly_pivot.index,
                title="🔥 Revenue Heatmap (Monthly)",
                color_continuous_scale='viridis',
                aspect='auto'
            )
            fig_heatmap.update_layout(
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)',
                font=dict(color='white'),
                title_font_size=16,
                xaxis_title="Month",
                yaxis_title="Year"
            )
        else:
            # Create empty heatmap if no data
            fig_heatmap = go.Figure()
            fig_heatmap.update_layout(
                title="🔥 Revenue Heatmap (Monthly) - No Data",
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)',
                font=dict(color='white'),
                title_font_size=16
            )

        return fig_trend, fig_country, fig_products, fig_heatmap

    except Exception as e:
        st.error(f"Error creating charts: {e}")
        return None, None, None, None

def main_dashboard():
    """Main dashboard interface."""

    # Header
    st.markdown(f"""
        <div class="main-header">
            <h1>🛍️ E-commerce Analytics Dashboard</h1>
            <p>Welcome back, <strong>{st.session_state.username}</strong> • Real-time insights into your business performance</p>
        </div>
    """, unsafe_allow_html=True)

    # Initialize database (replace with your actual implementation)
    warehouse, etl, queries = init_database()

    # Load sample data for demo
    sales_data, country_data, product_data = load_sample_data()

    # Sidebar filters
    with st.sidebar:
        st.markdown("### 🎛️ Dashboard Filters")

        # Date range filter
        st.markdown('<div class="filter-container">', unsafe_allow_html=True)
        st.subheader("📅 Date Range")
        date_range = st.date_input(
            "Select date range",
            value=(datetime(2024, 1, 1), datetime(2024, 12, 31)),
            min_value=datetime(2023, 1, 1),
            max_value=datetime(2024, 12, 31)
        )

        # Ensure date_range is always a tuple
        if not isinstance(date_range, tuple):
            if hasattr(date_range, '__iter__') and len(date_range) == 2:
                date_range = tuple(date_range)
            else:
                date_range = (datetime(2024, 1, 1), datetime(2024, 12, 31))
        elif len(date_range) == 1:
            date_range = (date_range[0], date_range[0])

        st.markdown('</div>', unsafe_allow_html=True)

        # Country filter
        st.markdown('<div class="filter-container">', unsafe_allow_html=True)
        st.subheader("🌍 Countries")
        available_countries = country_data['country'].tolist()
        selected_countries = st.multiselect(
            "Select countries",
            available_countries,
            default=available_countries[:5]
        )
        st.markdown('</div>', unsafe_allow_html=True)

        # Refresh button
        if st.button("🔄 Refresh Data", use_container_width=True):
            st.cache_data.clear()
            st.rerun()

        # Logout button
        if st.button("🚪 Logout", use_container_width=True):
            st.session_state.authenticated = False
            st.session_state.username = ""
            st.rerun()

    # Key Metrics Row
    st.markdown("### 📊 Key Performance Indicators")

    # Filter sales data for metrics
    if len(date_range) == 2:
        start_date = pd.to_datetime(date_range[0])
        end_date = pd.to_datetime(date_range[1])
        filtered_sales = sales_data[
            (sales_data['date'] >= start_date) &
            (sales_data['date'] <= end_date)
        ]
    else:
        filtered_sales = sales_data

    col1, col2, col3, col4 = st.columns(4)

    with col1:
        total_revenue = filtered_sales['revenue'].sum()
        create_metric_card("Total Revenue", total_revenue, change=12.5, format_func=lambda x: f"${x:,.0f}")

    with col2:
        total_orders = filtered_sales['orders'].sum()
        create_metric_card("Total Orders", total_orders, change=8.3)

    with col3:
        avg_order_value = total_revenue / total_orders if total_orders > 0 else 0
        create_metric_card("Avg Order Value", avg_order_value, change=-2.1, format_func=lambda x: f"${x:.0f}")

    with col4:
        total_customers = filtered_sales['customers'].sum()
        create_metric_card("Total Customers", total_customers, change=15.7)

    # Charts Section
    if selected_countries:
        charts = create_charts(sales_data, country_data, product_data, date_range, selected_countries)

        if all(chart is not None for chart in charts):
            fig_trend, fig_country, fig_products, fig_heatmap = charts

            # First row of charts
            col1, col2 = st.columns([2, 1])

            with col1:
                st.markdown('<div class="chart-container">', unsafe_allow_html=True)
                st.plotly_chart(fig_trend, use_container_width=True)
                st.markdown('</div>', unsafe_allow_html=True)

            with col2:
                st.markdown('<div class="chart-container">', unsafe_allow_html=True)
                st.plotly_chart(fig_products, use_container_width=True)
                st.markdown('</div>', unsafe_allow_html=True)

            # Second row of charts
            col1, col2 = st.columns(2)

            with col1:
                st.markdown('<div class="chart-container">', unsafe_allow_html=True)
                st.plotly_chart(fig_country, use_container_width=True)
                st.markdown('</div>', unsafe_allow_html=True)

            with col2:
                st.markdown('<div class="chart-container">', unsafe_allow_html=True)
                st.plotly_chart(fig_heatmap, use_container_width=True)
                st.markdown('</div>', unsafe_allow_html=True)
        else:
            st.error("❌ Unable to generate charts. Please check your data and try again.")

        # Data Tables Section
        st.markdown("### 📋 Detailed Analytics")

        tab1, tab2, tab3 = st.tabs(["🏆 Top Products", "🌍 Country Performance", "📈 Sales Data"])

        with tab1:
            st.markdown('<div class="chart-container">', unsafe_allow_html=True)
            st.dataframe(
                product_data.head(10).style.format({
                    'revenue': '${:,.0f}',
                    'quantity': '{:,}'
                }),
                use_container_width=True
            )
            st.markdown('</div>', unsafe_allow_html=True)

        with tab2:
            st.markdown('<div class="chart-container">', unsafe_allow_html=True)
            filtered_country_display = country_data[country_data['country'].isin(selected_countries)]
            st.dataframe(
                filtered_country_display.style.format({
                    'revenue': '${:,.0f}',
                    'customers': '{:,}'
                }),
                use_container_width=True
            )
            st.markdown('</div>', unsafe_allow_html=True)

        with tab3:
            st.markdown('<div class="chart-container">', unsafe_allow_html=True)
            st.dataframe(
                filtered_sales.tail(20).style.format({
                    'revenue': '${:,.0f}',
                    'orders': '{:,}',
                    'customers': '{:,}'
                }),
                use_container_width=True
            )
            st.markdown('</div>', unsafe_allow_html=True)

    else:
        st.warning("⚠️ Please select at least one country to display charts.")

def main():
    """Main application entry point."""

    # Load custom CSS
    load_css()

    # Initialize authentication
    init_auth()

    # Check authentication
    if not st.session_state.authenticated:
        login_page()
    else:
        main_dashboard()

if __name__ == "__main__":
    main()

Overwriting app.py


In [29]:
!pkill ngrok
!ngrok config add-authtoken 32TBaW0daeIghPAn9337TQk7zsT_4Py9Yw2LxAmSN9iXmrqBR

# Start Streamlit on port 8501
from pyngrok import ngrok
import time
ngrok.kill()  # Kill any old tunnels
time.sleep(1)
public_url = ngrok.connect(8501)
print("Public URL:", public_url)

# Run the app in the background
!streamlit run app.py --server.port 8501

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://15c7021ffbbf.ngrok-free.app" -> "http://localhost:8501"
📥 Downloading E-commerce datasets...


✅ Retail data: (541909, 8)

📋 Sample data preview:

--- RETAIL ---
  InvoiceNo StockCode  ... CustomerID         Country
0    536365    85123A  ...    17850.0  United Kingdom
1    536365     71053  ...    17850.0  United Kingdom
2    536365    84406B  ...    17850.0  United Kingdom

[3 rows x 8 columns]
Shape: (541909, 8)
🏭 Initializing E-commerce Dimensional Model...
✅ Tables created successfully!
Traceback (most recent call last):
  File "/content/app.py", line 3, in <module>
    from schema import *
  File "/content/schema.py", line 88, in <module>
    warehouse.get_table_info()
  File "/content/schema.py", line 76, in get_table_info
    inspector = inspect(create_engine(f'sqlite:///{self.db_path}'))
                ^^^^^^^
NameError: name 'inspect' is not defined. Did you mean: 'inspector'? Or did you forget to import 'inspect'?
